
byte pair embeddings https://github.com/bheinzerling/bpemb
biLASTM-CRF in tensor flow https://blog.dominodatalab.com/named-entity-recognition-ner-challenges-and-model
Attention address parsing https://doi.org/10.1145/3404835.3462904
Character level embedding in keras https://towardsdatascience.com/character-level-cnn-with-keras-50391c3adf33


In [18]:
import pandas as pd
import numpy as np
import re
import pylibpostal

In [17]:
pip install pylibpostal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: six
    Found existing installation: six 1.14.0
    Uninstalling six-1.14.0:
      Successfully uninstalled six-1.14.0
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from deepparse.parser import AddressParser

address_parser = AddressParser(model_type="bpemb", device=0)
address_parser_att = AddressParser(model_type="bpemb", attention_mechanism=True)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 34: CUDA driver is a stub library (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
/usr/local/lib/python3.8/dist-packages/deepparse/parser/address_parser.py:753: UserWarning: No CUDA device detected, device will be set to 'CPU'.
  warnings.warn("No CUDA device detected, device will be set to 'CPU'.")


Loading the embeddings model
downloading https://nlp.h-its.org/bpemb/multi/multi.wiki.bpe.vs100000.model


100%|██████████| 1965223/1965223 [00:00<00:00, 7402400.03B/s]


downloading https://nlp.h-its.org/bpemb/multi/multi.wiki.bpe.vs100000.d300.w2v.bin.tar.gz


100%|██████████| 112202964/112202964 [00:12<00:00, 9149983.10B/s]


/usr/local/lib/python3.8/dist-packages/deepparse/parser/address_parser.py:753: UserWarning: No CUDA device detected, device will be set to 'CPU'.
  warnings.warn("No CUDA device detected, device will be set to 'CPU'.")


Loading the embeddings model


# Create example addresses

This block is designed to give a structured example of how the parser's fail.

In [13]:
flat_type = "Flat"
flat_number_1 = "8"
flat_number_2 = "C"
building_name = "Easting House"
street_number = "23"
street_number_2 = "23-26"
street_name = "Broadfield Road"
postcode = "SW56 1ZZ"
city = "London"
country = "United Kingdom"


In [14]:

house_1 = ', '.join([f'{street_number} {street_name}', postcode, city, country])
house_2 = ', '.join([f'{street_number} {street_name}', postcode, city])
house_3 = ', '.join([f'{street_number} {street_name}', city])
flat_1 = ', '.join([f'{flat_type} {flat_number_1}', f'{street_number} {street_name}', postcode, city, country])
flat_2 = ', '.join([flat_number_1, f'{street_number} {street_name}', postcode, city, country])
flat_3 = ', '.join([f'{flat_number_1} {building_name}', f'{street_number} {street_name}', postcode, city, country])
flat_3 = ', '.join([f'{flat_number_1} {building_name}', f'{street_number_2} {street_name}', postcode, city, country])
flat_4 = ', '.join([f'{flat_number_1} {building_name}', street_name, postcode, city, country])

address_list = [house_1, house_2, house_3, flat_1, flat_2, flat_3, flat_4]

In [15]:
pd.DataFrame(address_list)

,0
0,"23 Broadfield Road, SW56 1ZZ, London, United K..."
1,"23 Broadfield Road, SW56 1ZZ, London"
2,"23 Broadfield Road, London"
3,"Flat 8, 23 Broadfield Road, SW56 1ZZ, London, ..."
4,"8, 23 Broadfield Road, SW56 1ZZ, London, Unite..."
5,"8 Easting House, 23-26 Broadfield Road, SW56 1..."
6,"8 Easting House, Broadfield Road, SW56 1ZZ, Lo..."


In [16]:
parsed_address_list = address_parser(address_list)
address_errors = pd.DataFrame([x.to_dict() for x in parsed_address_list]).drop(
    ["Orientation", "GeneralDelivery", "EOS"], axis  = 1)
address_errors

,StreetNumber,StreetName,Unit,Municipality,Province,PostalCode
0,23,broadfield road,None,london united,kingdom,sw56 1zz
1,23 sw56,broadfield road,1zz london,None,None,None
2,23,broadfield road,None,london,None,None
3,sw56,road,flat 8 23 broadfield,london,united kingdom,1zz
4,8 road,23 broadfield sw56,None,london united,kingdom,1zz
5,8,easting house,23-26 broadfield road sw56,london,united kingdom,1zz
6,8,easting house broadfield,road sw56,london united,kingdom,1zz


As can be seen above, the basic parser is not very good

Hopwever, whwen attention is used quality improves a lot

In [12]:
parsed_address_list_att = address_parser_att(address_list)
address_errors = pd.DataFrame([x.to_dict() for x in parsed_address_list_att]).drop(
    ["Orientation", "GeneralDelivery", "EOS"], axis  = 1)
address_errors

,StreetNumber,StreetName,Unit,Municipality,Province,PostalCode
0,23,broadfield road,None,london,england,sw56 1zz
1,23,broadfield road,None,None,london,sw56 1zz
2,23,None,None,road,london,broadfield
3,23,broadfield road,flat 8,london,england,sw56 1zz
4,8 23,broadfield road,None,london,england,sw56 1zz
5,8,easting house 23-26 broadfield road,None,london,england,sw56 1zz
6,8,easting house broadfield road,None,london,england,sw56 1zz


## VOA column headers

I can't find the VOA column headers so have had to take them from the data description pdf [here](https://voaratinglists.blob.core.windows.net/html/documents/2017%20Compiled%20List%20and%20SMV%20Data%20Specification.pdf)

In [2]:
VOA_headers_raw= ["Incrementing Entry Number", "Billing Authority Code", "NDR Community Code", 
 "BA Reference Number", "Primary And Secondary Description Code", "Primary Description Text",
"Unique Address Reference Number UARN", "Full Property Identifier", "Firms Name", "Number Or Name",
"Street", "Town", "Postal District", "County", "Postcode", "Effective Date", "Composite Indicator",
 "Rateable Value", "Appeal Settlement Code", "Assessment Reference", "List Alteration Date", "SCAT Code And Suffix",
 "Sub Street level 3", "Sub Street level 2", "Sub Street level 1", "Case Number", 
 "Current From Date", "Current To Date", 
]

#set to lower and replace spaces with underscore to turn the names into appropriate column names
VOA_headers = [x.lower().replace(" ", "_") for x in VOA_headers_raw]


Using the attention mechanism dramatically improves the quality of the results, but there are still some substantial issues

In [3]:
voa_businesses =  pd.read_csv('/tf/empty_homes_data/' +
                    'uk-englandwales-ndr-2017-listentries-compiled-epoch-0029-baseline-csv.csv',
                   sep = "*",
                   encoding_errors= 'ignore',
                    header=None,
                   names = VOA_headers[1:],
                    usecols = list(range(1,28))
                   )
voa_businesses['postcode'] = voa_businesses['postcode'].str.upper()

<ipython-input-3-0afde0e66eed>:1: DtypeWarning: Columns (1,2,6,12,13,17,22) have mixed types. Specify dtype option on import or set low_memory=False.
  voa_businesses =  pd.read_csv('/tf/empty_homes_data/' +


In [4]:
voa_businesses.iloc[:, 6:14]

,full_property_identifier,firms_name,number_or_name,street,town,postal_district,county,postcode
0,"MEDICAL CENTRE, ABBEYFIELDS, ELSTOW, BEDFORD",NaN,MEDICAL CENTRE,ABBEYFIELDS,BEDFORD,ELSTOW,NaN,MK42 9GP
1,"ELSTOW SCHOOL, ABBEYFIELDS, ELSTOW, BEDFORD",NaN,ELSTOW SCHOOL,ABBEYFIELDS,BEDFORD,ELSTOW,NaN,MK42 9GP
2,"5, ADAMS CLOSE, KEMPSTON, BEDFORD",NaN,5,ADAMS CLOSE,BEDFORD,KEMPSTON,NaN,MK42 7JE
3,"WOBURN HOUSE 3, ADAMS CLOSE, KEMPSTON, BEDFORD",NaN,WOBURN HOUSE 3,ADAMS CLOSE,BEDFORD,KEMPSTON,NaN,MK42 7JE
4,"1, ADAMS CLOSE, KEMPSTON, BEDFORD",NaN,1,ADAMS CLOSE,BEDFORD,KEMPSTON,NaN,MK42 7JE
...,...,...,...,...,...,...,...,...
2553594,"OFFICES 6-8 1ST FLOOR, WEAVERS BUSINESS CENTR...",NaN,OFFICES 6-8 1ST FLOOR,HAMLET ROAD,HAVERHILL,NaN,SUFFOLK,CB9 8EE
2553595,"ROOMS G/H, HOSTE HOUSE REFORMED CHURCH, WHITIN...",NaN,ROOMS G/H,WHITING STREET,BURY ST EDMUNDS,NaN,SUFFOLK,IP33 1NR
2553596,"UNIT 6, MENTA BUSINESS CENTRE 21-27, HOLLANDS ...",NaN,UNIT 6,HOLLANDS ROAD,HAVERHILL,NaN,SUFFOLK,CB9 8PU
2553597,"UNIT 5, MENTA BUSINESS CENTRE 21-27, HOLLANDS ...",NaN,UNIT 5,HOLLANDS ROAD,HAVERHILL,NaN,SUFFOLK,CB9 8PU


In [5]:
address_list = voa_businesses['full_property_identifier'].tolist()[0:100]
parsed_address_list_att = address_parser_att(address_list)

NameError: name 'address_parser_att' is not defined

In [112]:
address_errors = pd.DataFrame([x.to_dict() for x in parsed_address_list_att]).drop(
    ["Orientation", "GeneralDelivery", "EOS"], axis  = 1)
address_errors["address"] = address_list
address_errors

,StreetNumber,StreetName,Unit,Municipality,Province,PostalCode,address
0,None,medical,None,abbeyfields elstow,bedford,centre,"MEDICAL CENTRE, ABBEYFIELDS, ELSTOW, BEDFORD"
1,None,elstow,None,abbeyfields elstow,bedford,school,"ELSTOW SCHOOL, ABBEYFIELDS, ELSTOW, BEDFORD"
2,5,adams,None,kempston,bedford,close,"5, ADAMS CLOSE, KEMPSTON, BEDFORD"
3,None,woburn,None,adams close kempston,bedford,house 3,"WOBURN HOUSE 3, ADAMS CLOSE, KEMPSTON, BEDFORD"
4,1,adams,None,kempston,bedford,close,"1, ADAMS CLOSE, KEMPSTON, BEDFORD"
...,...,...,...,...,...,...,...
95,3g,hutchison,None,anderson cars alington road eynesbury st neots,cambs,site mk0225 at,"HUTCHISON 3G SITE MK0225 AT ANDERSON CARS, ALI..."
96,None,r/o,None,ampthill road,bedford,28a,"R/O 28A, AMPTHILL ROAD, BEDFORD"
97,36,None,None,road,bedford,ampthill,"36, AMPTHILL ROAD, BEDFORD"
98,35-37,None,None,road,bedford,ampthill,"35-37, AMPTHILL ROAD, BEDFORD"


## Find business counts per postcode

In [6]:
ocod_data =  pd.read_csv('/tf/empty_homes_data/' +
                    'OCOD_FULL_2022_02.csv',
                   encoding_errors= 'ignore').rename(columns = lambda x: x.lower().replace(" ", "_"))
ocod_data['postcode'] = ocod_data['postcode'].str.upper()
#ocod_data['postcode'] = ocod_data['postcode'].str.replace(" ", "")                 

<ipython-input-6-fb700e0752b6>:1: DtypeWarning: Columns (24,28,30,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  ocod_data =  pd.read_csv('/tf/empty_homes_data/' +


In [113]:
address_list = ocod_data['property_address'].tolist()[0:100]
parsed_address_list_att = address_parser_att(address_list)
address_errors = pd.DataFrame([x.to_dict() for x in parsed_address_list_att]).drop(
    ["Orientation", "GeneralDelivery", "EOS"], axis  = 1)
address_errors["address"] = address_list
address_errors

Vectorizing the address


,StreetNumber,StreetName,Unit,Municipality,Province,PostalCode,address
0,None,westleigh lodge care home nel,None,pan lane,leigh,(wn7 5jt),"Westleigh Lodge Care Home, Nel Pan Lane, Leigh..."
1,1a,canal,flat 1,street,manchester,(m1 3he),"Flat 1, 1a Canal Street, Manchester (M1 3HE)"
2,1,regent,flat 201,road,manchester,(m3 4ay),"Flat 201, 1 Regent Road, Manchester (M3 4AY)"
3,2a,gerard street,land at,ashton in makerfield,wigan,(wn4 9aa),"Land at 2a Gerard Street, Ashton In Makerfield..."
4,unit 111,timber wharf worsley,None,street,manchester,(m15 4nz),"Unit 111, Timber Wharf, Worsley Street, Manche..."
...,...,...,...,...,...,...,...
95,None,land on south side,None,darlington road,hartburn,of,"land on south side of Darlington Road, Hartburn"
96,None,land,None,britannia road,stockton-on-tees,at,"land at Britannia Road, Stockton-On-Tees"
97,site bt50/97,teesside industrial estate,None,thornaby,stockton-on-tees,(ts17 9jy),"Site BT50/97 Teesside Industrial Estate, Thorn..."
98,None,hill street,None,centre,middlesbrough,(ts1 1su),"Hill Street Centre, Middlesbrough (TS1 1SU)"


In [54]:
import io
import zipfile


post_area_id = 'AL'

target_post_area = "Data/multi_csv/ONSPD_NOV_2021_UK_" + post_area_id +".csv"

with zipfile.ZipFile("/tf/empty_homes_data/" + "ONSPD_NOV_2021_UK.zip") as zf:
    with io.TextIOWrapper(zf.open(target_post_area), encoding="utf-8") as f:
        ONSPD = pd.read_csv(f)

temp_postcode_matches = postcode_matches[postcode_matches['post_area']==post_area_id]

temp_postcode_matches   


,postcode,business_counts,ocod_counts,post_area
42,AL1 1AW,1.0,NaN,AL
43,AL1 1BX,7.0,NaN,AL
44,AL1 1BY,4.0,NaN,AL
45,AL1 1BZ,21.0,NaN,AL
46,AL1 1DQ,2.0,NaN,AL
...,...,...,...,...
470107,AL8 6UB,NaN,1.0,AL
470108,AL9 5RH,NaN,1.0,AL
470109,AL9 7QG,NaN,1.0,AL
470110,AL9 7RX,NaN,1.0,AL


In [9]:
ocod_data

NameError: name 'ocod_data' is not defined